# Assignment 2, Task B
### Ralph Gailis
### Student number:  28838750

## MongoDB Data Model

### Task B.1
We have two collections:
1. *Climate*: This has a single record for every day, and has a fairly continuous set of dates.
2. *Fire*: This has many records for every day, but only has records for a sparse number of dates.

In considering a data model for these collections, the most important factor is that *Date* is the common field for each collection, and so there is the possibility to join over this field.
We generally have 3 options to consider for a data model with a many-to-one relation over a field:
1. Embedded documents: We place the many *Fire* documents for each *Date* as an array of embedded documents in a given *Climate* document.
2. Child references: We keep an array of indices of the *Fire* documents as a field within each *Climate* document (i.e. one array of indices for each *Date*), and the *Fire* collection remains separate.  This enables fairly fast lookup of *Fire* data for a given day.
3. Parent reference: We keep both collections separate, and maintain a field in each *Fire* document whose value is an index to the associated (by *Date*) *Climate* document.

For the current application, we choose the first option.  This option is generally best if a few embedded (*Fire*) documents are anticipated for each given parent (*Climate*) document. By having documents embedded, the best query performance is achieved, as no outer joins are required to link two collections. (Joins are generally considered the most expensive query operation.)  If many (e.g. thousands) of documents are anticipated as child doucments, the second option would be considered better, as the array of embedded documents may grow without bound and cause memory overflow issues and excessively large documents.  If a very large (e.g. millions) of documents are anticipated, the third option is preferred, as the index array in the second option could grow without bound.

For the current applications, given we are interested in collecting fire data for each day, we do not anticipate that a situation would ever arise where thousands (let alone millions) of reports would be generated on a given day, hence the first approach, using embedded documents, will provide the best performance. An example of combining the two collections into a single collection containing embedded documents is provided below.  The *Fire* data for each day is contained in an array of embedded documents.

### Task B.2
Write a python program that will read the data from *Fire data-Part1* and *Climate 
data - Part1* and load them to the new database, using the data model described above.

We first import the relevant Python libraries, and read in the CSV data using Pandas.

In [1]:
import pandas as pd  # to load data and manipulate records
import numpy as np   # to handle pandas data types
import pymongo
from pymongo import MongoClient
from pprint import pprint  # for printing out MongoDB documents

# load csv data into data frames
climateDf = pd.read_csv('ClimateData-Part1.csv')
fireDf = pd.read_csv('FireData-Part1.csv')

In [2]:
# remove whitespace from Precipitation column name
climateDf.rename(columns={'Precipitation ': 'Precipitation'}, inplace=True)

display(climateDf.head())
print(climateDf.info())

,Station,Date,Air_Temperature_Celcius,Relative_Humidity,WindSpeed_knots,Max_Wind_Speed,Precipitation
0,948700,2016-12-31,19,56.8,7.9,11.1,0.00I
1,948700,2017-01-02,15,50.7,9.2,13.0,0.02G
2,948700,2017-01-03,16,53.6,8.1,15.0,0.00G
3,948700,2017-01-04,24,61.6,7.7,14.0,0.00I
4,948700,2017-01-05,24,62.3,7.0,13.0,0.00I


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 7 columns):
Station                    366 non-null int64
Date                       366 non-null object
Air_Temperature_Celcius    366 non-null int64
Relative_Humidity          366 non-null float64
WindSpeed_knots            366 non-null float64
Max_Wind_Speed             366 non-null float64
Precipitation              366 non-null object
dtypes: float64(3), int64(2), object(2)
memory usage: 20.1+ KB
None


In [3]:
display(fireDf.head())
print(fireDf.info())

,Latitude,Longitude,Surface_Temperature_Kelvin,Datetime,Power,Confidence,Date,Surface_Temperature_Celcius
0,-37.966,145.051,341.8,2017-12-27T04:16:51,26.7,78,2017-12-27,68
1,-35.541,143.311,336.3,2017-12-27T00:02:15,62.0,82,2017-12-27,63
2,-35.554,143.307,326.8,2017-12-27T00:02:15,23.8,67,2017-12-27,53
3,-35.543,143.316,340.4,2017-12-27T00:02:14,84.2,86,2017-12-27,67
4,-37.708,145.100,327.8,2017-12-25T04:29:08,16.2,80,2017-12-25,54


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2668 entries, 0 to 2667
Data columns (total 8 columns):
Latitude                       2668 non-null float64
Longitude                      2668 non-null float64
Surface_Temperature_Kelvin     2668 non-null float64
Datetime                       2668 non-null object
Power                          2668 non-null float64
Confidence                     2668 non-null int64
Date                           2668 non-null object
Surface_Temperature_Celcius    2668 non-null int64
dtypes: float64(4), int64(2), object(2)
memory usage: 166.8+ KB
None


The imported data looks to be in good order.  We now create a connection to the MongoDB database `fit5148_db`, in which we will store the climate and fire data.

In [4]:
# Connect to MongoDB on the default host and port, and open database
client = MongoClient () # connect on the default host and port
db = client.fit5148_db

# create a new mongodb collection "climate_fire"
climate_fire = db.climate_fire

We now define a helper function and then implement the code to write the fire and climate data into the MongoDB database using the data model defined above.

In [5]:
def df_to_dict(df, i):
    '''
    Convert a row of a data frame (a record) to a dictionary.
    
    Arguments:
    df -- a data frame
    i -- row number for the data frame
    
    Explanation:  A simple way to do this operation is to invoke
            > df.loc[i].to_dict()
        However, when this dictionary is passed to MongoDB, an error
        is returned because the dictionary values are numpy types.
        This function converts the numpy numeric types to standard Python
        types, and leaves strings unchanged.

    Return:
    record -- a dictionary with keys as the data frame column headers.
    '''
    
    record = df.loc[i].to_dict()  # extract record from row i
    
    # convert numeric values from numpy types to standard Python types
    record = {k: np.asscalar(v) if isinstance(v, np.generic) else v
                  for k, v in record.items()}
    return record

# row index for Fire dataframe - start at max because it is reverse sorted
j = len(fireDf) - 1

# storage for the documents to insert into MongoDB
climate_fire_list = []

# get first record of Fire dataframe as a dictionary
fireRecord = df_to_dict(fireDf, j)

# scan over each record in Climate dataframe
for i in climateDf.index:
    climRecord = df_to_dict(climateDf, i)  # construct next Climate document
    
    # search for matching dates between Climate and Fire
    while climRecord['Date'] == fireRecord['Date'] and j >= 0:
        if 'Fires' not in climRecord.keys():
            # initialise new array of embedded documents
            climRecord['Fires'] = []
            
        # insert Fire record as embedded document
        climRecord['Fires'].append(fireRecord)
        j -= 1
        if j < 0:  # end of Fire data frame
            break
        else:      # get next fire record
            fireRecord = df_to_dict(fireDf, j)
            
    # store next document in documents collection (list)
    climate_fire_list.append(climRecord)

climate_fire.insert_many(climate_fire_list)  # insert all documents into MongoDB


### Task B.3
Write queries to answer the Tasks A2-A8 on the new database.

In [6]:
# 2. Find climate data on 15th December 2017.
cursor = climate_fire.find({"Date": "2017-12-15"}, {"_id": 0, "Fires": 0})
for document in cursor: 
    pprint(document)

{'Air_Temperature_Celcius': 18,
 'Date': '2017-12-15',
 'Max_Wind_Speed': 14.0,
 'Precipitation': ' 0.00I',
 'Relative_Humidity': 52.0,
 'Station': 948702,
 'WindSpeed_knots': 7.1}


In [7]:
# 3. Find the latitude, longitude, surface temperature, and confidence when the surface 
#    temperature (°C) was between 65 °C and 100 °C.
cursor = climate_fire.aggregate([
    { "$unwind": "$Fires" },
    { "$match":
      { "Fires.Surface_Temperature_Celcius": {"$gte": 65, "$lte": 100} }
    },
    { "$project":
      { "_id": 0,
        "Fires.Latitude": 1,
        "Fires.Longitude": 1,
        "Fires.Surface_Temperature_Celcius": 1,
        "Fires.Confidence": 1
      }
    }
])
for document in cursor: 
    pprint(document)

{'Fires': {'Confidence': 97,
           'Latitude': -37.6572,
           'Longitude': 142.0703,
           'Surface_Temperature_Celcius': 80}}
{'Fires': {'Confidence': 94,
           'Latitude': -37.2284,
           'Longitude': 147.9187,
           'Surface_Temperature_Celcius': 73}}
{'Fires': {'Confidence': 85,
           'Latitude': -37.4128,
           'Longitude': 147.0242,
           'Surface_Temperature_Celcius': 98}}
{'Fires': {'Confidence': 80,
           'Latitude': -37.0055,
           'Longitude': 148.1582,
           'Surface_Temperature_Celcius': 68}}
{'Fires': {'Confidence': 100,
           'Latitude': -37.4229,
           'Longitude': 147.02700000000002,
           'Surface_Temperature_Celcius': 99}}
{'Fires': {'Confidence': 84,
           'Latitude': -37.0193,
           'Longitude': 148.1459,
           'Surface_Temperature_Celcius': 71}}
{'Fires': {'Confidence': 94,
           'Latitude': -34.3555,
           'Longitude': 141.5495,
           'Surface_Temperature_Cel

           'Latitude': -36.6986,
           'Longitude': 142.7259,
           'Surface_Temperature_Celcius': 72}}
{'Fires': {'Confidence': 100,
           'Latitude': -36.9018,
           'Longitude': 141.0146,
           'Surface_Temperature_Celcius': 100}}
{'Fires': {'Confidence': 94,
           'Latitude': -36.6349,
           'Longitude': 142.2125,
           'Surface_Temperature_Celcius': 86}}
{'Fires': {'Confidence': 90,
           'Latitude': -36.0387,
           'Longitude': 145.6787,
           'Surface_Temperature_Celcius': 67}}
{'Fires': {'Confidence': 92,
           'Latitude': -37.8161,
           'Longitude': 142.9286,
           'Surface_Temperature_Celcius': 71}}
{'Fires': {'Confidence': 97,
           'Latitude': -37.7617,
           'Longitude': 143.08100000000002,
           'Surface_Temperature_Celcius': 81}}
{'Fires': {'Confidence': 91,
           'Latitude': -37.3803,
           'Longitude': 145.611,
           'Surface_Temperature_Celcius': 68}}
{'Fires': {'Confi

{'Fires': {'Confidence': 89,
           'Latitude': -36.0295,
           'Longitude': 143.6409,
           'Surface_Temperature_Celcius': 65}}
{'Fires': {'Confidence': 92,
           'Latitude': -36.1924,
           'Longitude': 143.1716,
           'Surface_Temperature_Celcius': 70}}
{'Fires': {'Confidence': 100,
           'Latitude': -36.4142,
           'Longitude': 143.1077,
           'Surface_Temperature_Celcius': 97}}
{'Fires': {'Confidence': 93,
           'Latitude': -36.1941,
           'Longitude': 143.1604,
           'Surface_Temperature_Celcius': 72}}
{'Fires': {'Confidence': 96,
           'Latitude': -36.9498,
           'Longitude': 142.2012,
           'Surface_Temperature_Celcius': 78}}
{'Fires': {'Confidence': 100,
           'Latitude': -36.7957,
           'Longitude': 143.9652,
           'Surface_Temperature_Celcius': 87}}
{'Fires': {'Confidence': 99,
           'Latitude': -36.7943,
           'Longitude': 143.9571,
           'Surface_Temperature_Celcius': 84

           'Latitude': -36.6537,
           'Longitude': 143.8781,
           'Surface_Temperature_Celcius': 76}}
{'Fires': {'Confidence': 99,
           'Latitude': -36.5438,
           'Longitude': 144.7548,
           'Surface_Temperature_Celcius': 84}}
{'Fires': {'Confidence': 90,
           'Latitude': -36.094,
           'Longitude': 145.7646,
           'Surface_Temperature_Celcius': 67}}
{'Fires': {'Confidence': 93,
           'Latitude': -36.388000000000005,
           'Longitude': 141.8309,
           'Surface_Temperature_Celcius': 73}}
{'Fires': {'Confidence': 100,
           'Latitude': -36.0576,
           'Longitude': 141.6122,
           'Surface_Temperature_Celcius': 94}}
{'Fires': {'Confidence': 96,
           'Latitude': -36.0594,
           'Longitude': 141.5907,
           'Surface_Temperature_Celcius': 79}}
{'Fires': {'Confidence': 93,
           'Latitude': -36.4046,
           'Longitude': 141.1534,
           'Surface_Temperature_Celcius': 73}}
{'Fires': {'Confi

           'Surface_Temperature_Celcius': 90}}
{'Fires': {'Confidence': 96,
           'Latitude': -38.126999999999995,
           'Longitude': 143.82,
           'Surface_Temperature_Celcius': 77}}
{'Fires': {'Confidence': 99,
           'Latitude': -37.293,
           'Longitude': 141.245,
           'Surface_Temperature_Celcius': 85}}
{'Fires': {'Confidence': 100,
           'Latitude': -37.294000000000004,
           'Longitude': 141.232,
           'Surface_Temperature_Celcius': 87}}
{'Fires': {'Confidence': 91,
           'Latitude': -37.247,
           'Longitude': 141.278,
           'Surface_Temperature_Celcius': 68}}
{'Fires': {'Confidence': 92,
           'Latitude': -36.098,
           'Longitude': 143.74,
           'Surface_Temperature_Celcius': 67}}
{'Fires': {'Confidence': 90,
           'Latitude': -37.330999999999996,
           'Longitude': 143.122,
           'Surface_Temperature_Celcius': 65}}
{'Fires': {'Confidence': 97,
           'Latitude': -37.69,
           '

In [8]:
# 4. Find surface temperature (°C), air temperature (°C), relative humidity and maximum 
#    wind speed on 15th and 16th of December 2017.
cursor = climate_fire.aggregate([
    # select relevant records according to "Date" attribute
    { "$match": {"Date": {"$in": ["2017-12-15", "2017-12-16"]}}},
    # project the required attributes for display
    { "$project":
      { "_id": 0,
        "Date": 1,
        "Surface_Temperature_Celcius": "$Fires.Surface_Temperature_Celcius",
        "Air_Temperature_Celcius": 1,
        "Relative_Humidity": 1,
        "Max_Wind_Speed": 1
      }
    }
])
for document in cursor: 
    pprint(document)

{'Air_Temperature_Celcius': 18,
 'Date': '2017-12-15',
 'Max_Wind_Speed': 14.0,
 'Relative_Humidity': 52.0,
 'Surface_Temperature_Celcius': [40, 38, 36, 42]}
{'Air_Temperature_Celcius': 18,
 'Date': '2017-12-16',
 'Max_Wind_Speed': 13.0,
 'Relative_Humidity': 53.7,
 'Surface_Temperature_Celcius': [57,
                                 64,
                                 55,
                                 48,
                                 73,
                                 60,
                                 56,
                                 66,
                                 55,
                                 75,
                                 55,
                                 73,
                                 54,
                                 33,
                                 43]}


In [9]:
# 5. Find datetime, air temperature (°C), surface temperature (°C) and confidence when 
#    the confidence is between 80 and 100.
cursor = climate_fire.aggregate([
    { "$unwind": "$Fires" },
    { "$match":
      { "Fires.Confidence": {"$gte": 80, "$lte": 100} }
    },
    { "$project":
      { "_id": 0,
        "Fires.Datetime": 1,
        "Air_Temperature_Celcius": 1,
        "Fires.Surface_Temperature_Celcius": 1,
        "Fires.Confidence": 1
      }
    }
])
for document in cursor: 
    pprint(document)

{'Air_Temperature_Celcius': 20,
 'Fires': {'Confidence': 85,
           'Datetime': '2017-03-06T05:06:20',
           'Surface_Temperature_Celcius': 59}}
{'Air_Temperature_Celcius': 20,
 'Fires': {'Confidence': 87,
           'Datetime': '2017-03-06T05:06:30',
           'Surface_Temperature_Celcius': 62}}
{'Air_Temperature_Celcius': 19,
 'Fires': {'Confidence': 88,
           'Datetime': '2017-03-07T04:16:10',
           'Surface_Temperature_Celcius': 64}}
{'Air_Temperature_Celcius': 23,
 'Fires': {'Confidence': 86,
           'Datetime': '2017-03-09T13:23:40',
           'Surface_Temperature_Celcius': 41}}
{'Air_Temperature_Celcius': 19,
 'Fires': {'Confidence': 81,
           'Datetime': '2017-03-10T04:43:50',
           'Surface_Temperature_Celcius': 55}}
{'Air_Temperature_Celcius': 19,
 'Fires': {'Confidence': 97,
           'Datetime': '2017-03-10T04:45:30',
           'Surface_Temperature_Celcius': 80}}
{'Air_Temperature_Celcius': 19,
 'Fires': {'Confidence': 94,
           'Dat

{'Air_Temperature_Celcius': 18,
 'Fires': {'Confidence': 85,
           'Datetime': '2017-03-28T04:26:50',
           'Surface_Temperature_Celcius': 59}}
{'Air_Temperature_Celcius': 18,
 'Fires': {'Confidence': 97,
           'Datetime': '2017-03-28T04:26:50',
           'Surface_Temperature_Celcius': 81}}
{'Air_Temperature_Celcius': 18,
 'Fires': {'Confidence': 87,
           'Datetime': '2017-03-28T04:26:50',
           'Surface_Temperature_Celcius': 62}}
{'Air_Temperature_Celcius': 18,
 'Fires': {'Confidence': 100,
           'Datetime': '2017-03-28T04:26:50',
           'Surface_Temperature_Celcius': 90}}
{'Air_Temperature_Celcius': 18,
 'Fires': {'Confidence': 83,
           'Datetime': '2017-03-28T04:26:50',
           'Surface_Temperature_Celcius': 57}}
{'Air_Temperature_Celcius': 18,
 'Fires': {'Confidence': 80,
           'Datetime': '2017-03-28T04:26:50',
           'Surface_Temperature_Celcius': 53}}
{'Air_Temperature_Celcius': 18,
 'Fires': {'Confidence': 81,
           'Da

           'Surface_Temperature_Celcius': 58}}
{'Air_Temperature_Celcius': 16,
 'Fires': {'Confidence': 100,
           'Datetime': '2017-04-04T04:33:30',
           'Surface_Temperature_Celcius': 113}}
{'Air_Temperature_Celcius': 16,
 'Fires': {'Confidence': 81,
           'Datetime': '2017-04-04T04:33:40',
           'Surface_Temperature_Celcius': 54}}
{'Air_Temperature_Celcius': 16,
 'Fires': {'Confidence': 92,
           'Datetime': '2017-04-04T04:33:50',
           'Surface_Temperature_Celcius': 81}}
{'Air_Temperature_Celcius': 16,
 'Fires': {'Confidence': 84,
           'Datetime': '2017-04-04T04:34:00',
           'Surface_Temperature_Celcius': 57}}
{'Air_Temperature_Celcius': 16,
 'Fires': {'Confidence': 85,
           'Datetime': '2017-04-04T04:34:20',
           'Surface_Temperature_Celcius': 66}}
{'Air_Temperature_Celcius': 16,
 'Fires': {'Confidence': 96,
           'Datetime': '2017-04-04T04:34:20',
           'Surface_Temperature_Celcius': 78}}
{'Air_Temperature_Celcius':

           'Datetime': '2017-04-06T04:23:10',
           'Surface_Temperature_Celcius': 65}}
{'Air_Temperature_Celcius': 19,
 'Fires': {'Confidence': 93,
           'Datetime': '2017-04-06T04:23:30',
           'Surface_Temperature_Celcius': 72}}
{'Air_Temperature_Celcius': 19,
 'Fires': {'Confidence': 99,
           'Datetime': '2017-04-06T04:23:50',
           'Surface_Temperature_Celcius': 85}}
{'Air_Temperature_Celcius': 19,
 'Fires': {'Confidence': 92,
           'Datetime': '2017-04-06T04:23:50',
           'Surface_Temperature_Celcius': 70}}
{'Air_Temperature_Celcius': 19,
 'Fires': {'Confidence': 89,
           'Datetime': '2017-04-06T04:24:00',
           'Surface_Temperature_Celcius': 64}}
{'Air_Temperature_Celcius': 19,
 'Fires': {'Confidence': 89,
           'Datetime': '2017-04-06T04:24:20',
           'Surface_Temperature_Celcius': 65}}
{'Air_Temperature_Celcius': 19,
 'Fires': {'Confidence': 89,
           'Datetime': '2017-04-06T04:24:20',
           'Surface_Temperatur

{'Air_Temperature_Celcius': 16,
 'Fires': {'Confidence': 81,
           'Datetime': '2017-04-13T04:26:30',
           'Surface_Temperature_Celcius': 55}}
{'Air_Temperature_Celcius': 16,
 'Fires': {'Confidence': 81,
           'Datetime': '2017-04-13T04:26:30',
           'Surface_Temperature_Celcius': 54}}
{'Air_Temperature_Celcius': 16,
 'Fires': {'Confidence': 81,
           'Datetime': '2017-04-13T04:26:30',
           'Surface_Temperature_Celcius': 54}}
{'Air_Temperature_Celcius': 16,
 'Fires': {'Confidence': 91,
           'Datetime': '2017-04-13T04:26:30',
           'Surface_Temperature_Celcius': 68}}
{'Air_Temperature_Celcius': 16,
 'Fires': {'Confidence': 88,
           'Datetime': '2017-04-13T04:26:30',
           'Surface_Temperature_Celcius': 64}}
{'Air_Temperature_Celcius': 16,
 'Fires': {'Confidence': 83,
           'Datetime': '2017-04-13T04:26:30',
           'Surface_Temperature_Celcius': 56}}
{'Air_Temperature_Celcius': 16,
 'Fires': {'Confidence': 92,
           'Dat

...

           'Datetime': '2017-05-03T04:03:00',
           'Surface_Temperature_Celcius': 60}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 100,
           'Datetime': '2017-05-03T04:03:00',
           'Surface_Temperature_Celcius': 91}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 93,
           'Datetime': '2017-05-03T04:03:10',
           'Surface_Temperature_Celcius': 72}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 100,
           'Datetime': '2017-05-03T04:03:50',
           'Surface_Temperature_Celcius': 110}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 100,
           'Datetime': '2017-05-03T04:05:10',
           'Surface_Temperature_Celcius': 104}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 90,
           'Datetime': '2017-05-03T04:05:20',
           'Surface_Temperature_Celcius': 67}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 84,
           'Datetime': '2017-05-03T04:05:20',
           'Surface_Tempe

{'Air_Temperature_Celcius': 12,
 'Fires': {'Confidence': 93,
           'Datetime': '2017-05-08T04:26:50',
           'Surface_Temperature_Celcius': 72}}
{'Air_Temperature_Celcius': 12,
 'Fires': {'Confidence': 81,
           'Datetime': '2017-05-09T05:02:50',
           'Surface_Temperature_Celcius': 54}}
{'Air_Temperature_Celcius': 12,
 'Fires': {'Confidence': 81,
           'Datetime': '2017-05-09T12:56:40',
           'Surface_Temperature_Celcius': 39}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 99,
           'Datetime': '2017-05-10T04:08:10',
           'Surface_Temperature_Celcius': 85}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 80,
           'Datetime': '2017-05-10T04:08:10',
           'Surface_Temperature_Celcius': 53}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 85,
           'Datetime': '2017-05-10T04:08:10',
           'Surface_Temperature_Celcius': 59}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 87,
           'Dat

           'Surface_Temperature_Celcius': 65}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 100,
           'Datetime': '2017-05-15T04:26:20',
           'Surface_Temperature_Celcius': 110}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 94,
           'Datetime': '2017-05-15T04:26:20',
           'Surface_Temperature_Celcius': 74}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 90,
           'Datetime': '2017-05-15T04:26:20',
           'Surface_Temperature_Celcius': 67}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 86,
           'Datetime': '2017-05-15T04:26:20',
           'Surface_Temperature_Celcius': 61}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 83,
           'Datetime': '2017-05-15T04:26:20',
           'Surface_Temperature_Celcius': 56}}
{'Air_Temperature_Celcius': 10,
 'Fires': {'Confidence': 82,
           'Datetime': '2017-05-15T04:26:20',
           'Surface_Temperature_Celcius': 55}}
{'Air_Temperature_Celcius':

           'Surface_Temperature_Celcius': 52}}
{'Air_Temperature_Celcius': 17,
 'Fires': {'Confidence': 100,
           'Datetime': '2017-09-23T12:47:12',
           'Surface_Temperature_Celcius': 48}}
{'Air_Temperature_Celcius': 17,
 'Fires': {'Confidence': 100,
           'Datetime': '2017-09-23T12:47:12',
           'Surface_Temperature_Celcius': 65}}
{'Air_Temperature_Celcius': 17,
 'Fires': {'Confidence': 100,
           'Datetime': '2017-09-23T12:47:12',
           'Surface_Temperature_Celcius': 56}}
{'Air_Temperature_Celcius': 17,
 'Fires': {'Confidence': 86,
           'Datetime': '2017-09-23T23:50:44',
           'Surface_Temperature_Celcius': 56}}
{'Air_Temperature_Celcius': 14,
 'Fires': {'Confidence': 84,
           'Datetime': '2017-09-24T04:04:04',
           'Surface_Temperature_Celcius': 52}}
{'Air_Temperature_Celcius': 14,
 'Fires': {'Confidence': 86,
           'Datetime': '2017-09-24T04:04:18',
           'Surface_Temperature_Celcius': 76}}
{'Air_Temperature_Celcius'

           'Surface_Temperature_Celcius': 53}}
{'Air_Temperature_Celcius': 24,
 'Fires': {'Confidence': 87,
           'Datetime': '2017-11-13T03:52:15',
           'Surface_Temperature_Celcius': 61}}
{'Air_Temperature_Celcius': 28,
 'Fires': {'Confidence': 87,
           'Datetime': '2017-11-14T04:35:04',
           'Surface_Temperature_Celcius': 65}}
{'Air_Temperature_Celcius': 24,
 'Fires': {'Confidence': 85,
           'Datetime': '2017-11-21T00:27:21',
           'Surface_Temperature_Celcius': 59}}
{'Air_Temperature_Celcius': 23,
 'Fires': {'Confidence': 88,
           'Datetime': '2017-11-22T03:46:00',
           'Surface_Temperature_Celcius': 64}}
{'Air_Temperature_Celcius': 23,
 'Fires': {'Confidence': 84,
           'Datetime': '2017-11-22T03:46:00',
           'Surface_Temperature_Celcius': 59}}
{'Air_Temperature_Celcius': 24,
 'Fires': {'Confidence': 93,
           'Datetime': '2017-11-23T04:28:59',
           'Surface_Temperature_Celcius': 73}}
{'Air_Temperature_Celcius': 2

In [10]:
# 6. Find top 10 records with highest surface temperature (°C).
cursor = climate_fire.aggregate([
    { "$match": {"Fires.Surface_Temperature_Celcius": {"$exists": True}} },
    { "$unwind": "$Fires" },
    { "$sort": {"Fires.Surface_Temperature_Celcius": pymongo.DESCENDING} },
    { "$limit": 10}     
])
for document in cursor:
    pprint(document)

{'Air_Temperature_Celcius': 15,
 'Date': '2017-04-18',
 'Fires': {'Confidence': 100,
           'Date': '2017-04-18',
           'Datetime': '2017-04-18T04:52:00',
           'Latitude': -38.1665,
           'Longitude': 143.062,
           'Power': 239.8,
           'Surface_Temperature_Celcius': 124,
           'Surface_Temperature_Kelvin': 397.5},
 'Max_Wind_Speed': 9.9,
 'Precipitation': ' 0.00I',
 'Relative_Humidity': 56.1,
 'Station': 948701,
 'WindSpeed_knots': 5.1,
 '_id': ObjectId('5bb346c09343690faf2eb9be')}
{'Air_Temperature_Celcius': 16,
 'Date': '2017-04-04',
 'Fires': {'Confidence': 100,
           'Date': '2017-04-04',
           'Datetime': '2017-04-04T04:32:50',
           'Latitude': -36.343,
           'Longitude': 142.1986,
           'Power': 233.4,
           'Surface_Temperature_Celcius': 123,
           'Surface_Temperature_Kelvin': 396.3},
 'Max_Wind_Speed': 12.0,
 'Precipitation': ' 0.00I',
 'Relative_Humidity': 47.5,
 'Station': 948701,
 'WindSpeed_knots': 5.

In [11]:
# 7. Find the number of fire in each day. You are required to only display the total 
#    number of fire and the date in the output.
cursor = climate_fire.aggregate([
    { "$unwind": "$Fires" },
    { "$group": {"_id": "$Date", "Num": {"$sum": 1}}},
    { "$sort" : { "_id" : 1 } },
    { "$project":
      { "_id" : 0,
        "Date": "$_id",
        "Number of fires": "$Num"
      }
    }
])
for document in cursor:
    pprint(document)

{'Date': '2017-03-06', 'Number of fires': 2}
{'Date': '2017-03-07', 'Number of fires': 1}
{'Date': '2017-03-08', 'Number of fires': 2}
{'Date': '2017-03-09', 'Number of fires': 3}
{'Date': '2017-03-10', 'Number of fires': 8}
{'Date': '2017-03-12', 'Number of fires': 5}
{'Date': '2017-03-13', 'Number of fires': 2}
{'Date': '2017-03-14', 'Number of fires': 10}
{'Date': '2017-03-15', 'Number of fires': 7}
{'Date': '2017-03-17', 'Number of fires': 6}
{'Date': '2017-03-18', 'Number of fires': 3}
{'Date': '2017-03-19', 'Number of fires': 21}
{'Date': '2017-03-24', 'Number of fires': 2}
{'Date': '2017-03-25', 'Number of fires': 13}
{'Date': '2017-03-26', 'Number of fires': 17}
{'Date': '2017-03-28', 'Number of fires': 54}
{'Date': '2017-03-29', 'Number of fires': 1}
{'Date': '2017-03-31', 'Number of fires': 22}
{'Date': '2017-04-01', 'Number of fires': 7}
{'Date': '2017-04-02', 'Number of fires': 5}
{'Date': '2017-04-03', 'Number of fires': 72}
{'Date': '2017-04-04', 'Number of fires': 89}
{'

In [13]:
# 8. Find the average surface temperature (°C) for each day. You are required to only 
#    display average surface temperature (°C) and the date in the output.
cursor = climate_fire.aggregate([
    { "$unwind": "$Fires" },
    { "$group": {"_id": "$Date", "Avg": {"$avg": "$Fires.Surface_Temperature_Celcius"}}},
    { "$sort" : { "_id" : 1 } },
    { "$project":
      { "_id" : 0,
        "Date": "$_id",
        "Average surface temperature (°C)": "$Avg"
      }
    }
])
for document in cursor:
    pprint(document)

{'Average surface temperature (°C)': 60.5, 'Date': '2017-03-06'}
{'Average surface temperature (°C)': 64.0, 'Date': '2017-03-07'}
{'Average surface temperature (°C)': 51.5, 'Date': '2017-03-08'}
{'Average surface temperature (°C)': 46.666666666666664, 'Date': '2017-03-09'}
{'Average surface temperature (°C)': 69.375, 'Date': '2017-03-10'}
{'Average surface temperature (°C)': 88.2, 'Date': '2017-03-12'}
{'Average surface temperature (°C)': 38.5, 'Date': '2017-03-13'}
{'Average surface temperature (°C)': 65.6, 'Date': '2017-03-14'}
{'Average surface temperature (°C)': 46.0, 'Date': '2017-03-15'}
{'Average surface temperature (°C)': 59.5, 'Date': '2017-03-17'}
{'Average surface temperature (°C)': 79.33333333333333, 'Date': '2017-03-18'}
{'Average surface temperature (°C)': 65.57142857142857, 'Date': '2017-03-19'}
{'Average surface temperature (°C)': 49.0, 'Date': '2017-03-24'}
{'Average surface temperature (°C)': 66.0, 'Date': '2017-03-25'}
{'Average surface temperature (°C)': 56.88235294